<a href="https://colab.research.google.com/github/poojavelu/AI-tools-for-systems-engineering/blob/main/Tabular_Data_Classification_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import sklearn as skl

from collections import Counter
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df = pd.read_csv("/content/drive/MyDrive/ISE244/training_wids2024C1.csv")

# Data Pre-Processing

In [ ]:
# Analyze the data read
df.shape

(12906, 83)

In [ ]:
df.head()

,patient_id,patient_race,payer_type,patient_state,patient_zip3,patient_age,patient_gender,bmi,breast_cancer_diagnosis_code,breast_cancer_diagnosis_desc,...,disabled,poverty,limited_english,commute_time,health_uninsured,veteran,Ozone,PM25,N02,DiagPeriodL90D
0,475714,NaN,MEDICAID,CA,924,84,F,NaN,C50919,Malignant neoplasm of unsp site of unspecified...,...,12.871429,22.542857,10.100000,27.814286,11.200000,3.500000,52.237210,8.650555,18.606528,1
1,349367,White,COMMERCIAL,CA,928,62,F,28.49,C50411,Malig neoplm of upper-outer quadrant of right ...,...,8.957576,10.109091,8.057576,30.606061,7.018182,4.103030,42.301121,8.487175,20.113179,1
2,138632,White,COMMERCIAL,TX,760,43,F,38.09,C50112,Malignant neoplasm of central portion of left ...,...,11.253333,9.663333,3.356667,31.394915,15.066667,7.446667,40.108207,7.642753,14.839351,1
3,617843,White,COMMERCIAL,CA,926,45,F,NaN,C50212,Malig neoplasm of upper-inner quadrant of left...,...,8.845238,8.688095,5.280952,27.561905,4.404762,4.809524,42.070075,7.229393,15.894123,0
4,817482,NaN,COMMERCIAL,ID,836,55,F,NaN,1749,"Malignant neoplasm of breast (female), unspeci...",...,15.276000,11.224000,1.946000,26.170213,12.088000,13.106000,41.356058,4.110749,11.722197,0


In [ ]:
#  Overview of each column values to understand the distribution of them
for column in df.columns:
  print(column)
  print(df[column].unique().shape)
  print(Counter(df[column]))
  print()

patient_id
(12906,)
Counter({475714: 1, 349367: 1, 138632: 1, 617843: 1, 817482: 1, 111545: 1, 914071: 1, 479368: 1, 994014: 1, 155485: 1, 875977: 1, 343914: 1, 266700: 1, 437659: 1, 880521: 1, 615208: 1, 856366: 1, 971531: 1, 279917: 1, 529840: 1, 366792: 1, 198037: 1, 643360: 1, 791301: 1, 487817: 1, 345047: 1, 618259: 1, 907418: 1, 908851: 1, 393934: 1, 872836: 1, 711220: 1, 224239: 1, 744114: 1, 785337: 1, 603170: 1, 434330: 1, 981240: 1, 499087: 1, 805610: 1, 661644: 1, 603406: 1, 810014: 1, 935195: 1, 363209: 1, 352645: 1, 108853: 1, 479402: 1, 441661: 1, 474989: 1, 333593: 1, 982442: 1, 452512: 1, 820148: 1, 880294: 1, 461108: 1, 231503: 1, 339920: 1, 480970: 1, 908063: 1, 802331: 1, 465804: 1, 310908: 1, 145457: 1, 727984: 1, 438221: 1, 593578: 1, 106657: 1, 932496: 1, 241308: 1, 216674: 1, 352379: 1, 551002: 1, 225718: 1, 555910: 1, 300511: 1, 660003: 1, 568373: 1, 168274: 1, 427003: 1, 543669: 1, 471641: 1, 274093: 1, 836128: 1, 805088: 1, 378657: 1, 949396: 1, 808143: 1, 720

In [ ]:
# We see there are several missing values in many columns. Let see if we can just drop all rows with any missing data.
# If we drop all rows with missing data, we are left with only 3 rows
# So, we need a way to impute values for the missing data
df.dropna(inplace=False).shape

(3, 83)

In [ ]:
# patient_id is unique for each patient and it does not carry meaningful information
# On the other end, patient_gender is always female
# So we can drop these two features
df.drop(columns=["patient_gender", "patient_id"], inplace=True)

In [ ]:
# Column with the target data
target_column = "DiagPeriodL90D"

# Split the remaining features into categorical and numerical features
selected_features_categorical = [
  'patient_race',
  'payer_type',
  'patient_state',
  'patient_zip3',
  'breast_cancer_diagnosis_code',
  'metastatic_cancer_diagnosis_code',
  'metastatic_first_novel_treatment',
  'metastatic_first_novel_treatment_type',
  'Region',
  'Division',
]

selected_features_numerical = [
  'patient_age', 'bmi',
  'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19',
  'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s',
  'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married',
  'widowed', 'family_size', 'family_dual_income',
  'income_household_median', 'income_household_under_5',
  'income_household_5_to_10', 'income_household_10_to_15',
  'income_household_15_to_20', 'income_household_20_to_25',
  'income_household_25_to_35', 'income_household_35_to_50',
  'income_household_50_to_75', 'income_household_75_to_100',
  'income_household_100_to_150', 'income_household_150_over',
  'income_household_six_figure', 'income_individual_median',
  'home_ownership', 'housing_units', 'home_value', 'rent_median',
  'rent_burden', 'education_less_highschool', 'education_highschool',
  'education_some_college', 'education_bachelors', 'education_graduate',
  'education_college_or_above', 'education_stem_degree',
  'labor_force_participation', 'unemployment_rate', 'self_employed', 'farmer',
  'race_white', 'race_black', 'race_asian', 'race_native', 'race_pacific',
  'race_other', 'race_multiple', 'hispanic', 'disabled', 'poverty',
  'limited_english', 'commute_time', 'health_uninsured', 'veteran',
  'Ozone', 'PM25', 'N02',
]

# The only remaining feature is breast_cancer_diagnosis_desc which is a text feature
# We could potentially extract some keywords/features from this.
# But we save them for later and ignore this feature for now
selected_features_text = [
  'breast_cancer_diagnosis_desc',
]

all_features = selected_features_numerical + selected_features_categorical

In [ ]:
df_selected = df[all_features + [target_column]]

In [ ]:
# Find the number of NaNs for each feature
df_selected.isna().sum()

,0
patient_age,0
bmi,8965
population,1
density,1
age_median,1
...,...
metastatic_first_novel_treatment,12882
metastatic_first_novel_treatment_type,12882
Region,52
Division,52


In [ ]:
# Split data into training and testing
X = df_selected.drop(target_column, axis=1)
y = df_selected[target_column]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0
)

In [ ]:
# For numerical features, fill missing values with mean and apply standard scaling
numerical_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="mean")),
    ("scaler", StandardScaler())
])

# For categorical featuers, apply one-hot encoding
categorical_transformer = Pipeline([
    ("encoder", OneHotEncoder(handle_unknown="ignore")),
])

# Combine transformers into a ColumnTransformer
preprocessor = ColumnTransformer([
    ("num", numerical_transformer, selected_features_numerical),
    ("cat", categorical_transformer, selected_features_categorical)
])

# Learn the transformation only on training data and apply the same transformation on test data
# This is to avoid data leakage during normalization
X_train_dt = preprocessor.fit_transform(X_train)
X_test_dt = preprocessor.transform(X_test)

# 1. Decision Trees

Decision trees inherently use feature selection at each node.
So, we need not do any feature selection and we let decision tree handle it by itself.

Similarly, it doesn't require feature scaling as well.
So, we do one-hot encoding for categorical features only.

In [ ]:
# Train decision tree classifier
# The hyperparameters were chosen by manually changing the values and picking the one that gave highest accuracy
decision_tree_classifier = DecisionTreeClassifier(min_samples_leaf=50, max_depth=15, random_state=0)
decision_tree_classifier.fit(X_train_dt, y_train)

DecisionTreeClassifier(max_depth=15, min_samples_leaf=50, random_state=0)

In [ ]:
y_pred = decision_tree_classifier.predict(X_test_dt)

In [ ]:
decision_tree_accuracy = accuracy_score(y_test, y_pred)

In [ ]:
decision_tree_accuracy

0.8032532920216886

In [ ]:
# Visualizing constructed decision tree

from sklearn import tree
import matplotlib.pyplot as plt


plt.figure(figsize=(120,80))
tree.plot_tree(
    decision_tree_classifier,
    filled=True,
    # feature_names=all_features,
)
plt.show()

# Feature Selection

* I take top 100 features using chi-squared function, utilizing SelectKBest function of sklearn
    * I arrived at 100 since that the accuracy stayed the same after further increase in number of features (for logistic regression)
* We also added feature scaling (in pre-processing) so its better for the classifiers to work on
* Feature selection and scaling are done based on the training data only the same transformations are repeated on the testing data. This avoids data leakage.

In [ ]:
selector = SelectKBest(k=100)
X_train_selected = selector.fit_transform(X_train_dt, y_train)
X_test_selected = selector.transform(X_test_dt)

# 2.  Logistic Regression

In [ ]:
logistic_regression_classifier = LogisticRegression(max_iter=100_000, penalty='l2', n_jobs=8)
logistic_regression_classifier.fit(X_train_selected, y_train)

LogisticRegression(max_iter=100000, n_jobs=8)

In [ ]:
y_pred = logistic_regression_classifier.predict(X_test_selected)

In [ ]:
logistic_regression_accuracy = accuracy_score(y_test, y_pred)
print(logistic_regression_accuracy)

0.8109992254066615


# 3. Random Forest

In [ ]:
random_forest_classifier = RandomForestClassifier(n_estimators=500, min_samples_leaf=25, max_depth=15, random_state=0)
random_forest_classifier.fit(X_train_selected, y_train)

RandomForestClassifier(max_depth=15, min_samples_leaf=25, n_estimators=500,
                       random_state=0)

In [ ]:
y_pred = random_forest_classifier.predict(X_test_selected)

In [ ]:
random_forest_accuracy = accuracy_score(y_test, y_pred)
print(random_forest_accuracy)

0.8051897753679318


# 4. Support Vector Machine

In [ ]:
svc_classifier = SVC(C=1.0, kernel='linear', gamma='scale', random_state=0)
svc_classifier.fit(X_train_selected, y_train)

SVC(kernel='linear', random_state=0)

In [ ]:
y_pred = svc_classifier.predict(X_test_selected)

In [ ]:
svm_forest_accuracy = accuracy_score(y_test, y_pred)
print(svm_forest_accuracy)

0.8113865220759101


# Conclusion

Based on the experiments, the accuracy obtained using the different algorithms are:


| ML Algorithm | Accuracy |
| --- | ----------- |
| Decision Tree | 80.33% |
| Logistic Regression | 81.10% |
| Random Forest | 80.52% |
| Support Vector Machine | 81.14% |


Based on this, we see that the SVM has the highest accuracy.

However, the accuracy of all the algorithms are very close to each other.


# Manual Feature Selection

Earlier, I did feature selection using `SelectKBest` function which uses mathematical formulations to select the best features.

As an alternate way, I want to add features one by one. The idea is:
1. Start with selected_features = [], remaining_fatures = all_features
2. Go over all features in remaining_fatures list one by one and compute the accuracy of the model if we were to add that particular feature to the selected_features and train the model with those features only.
3. Pick the feature, add it to selected_features and repeat Step 2
4. Break this loop when there is no improvement in accuracy by adding any more feature (or we have selected all features).

This code is currently incomplete and needs some work.

In [ ]:
all_features = ['patient_race', 'payer_type', 'patient_state',
       'patient_zip3', 'patient_age', 'bmi', 'breast_cancer_diagnosis_code',
       'breast_cancer_diagnosis_desc', 'metastatic_cancer_diagnosis_code',
       'metastatic_first_novel_treatment',
       'metastatic_first_novel_treatment_type', 'Region', 'Division',
       'population', 'density', 'age_median', 'age_under_10', 'age_10_to_19',
       'age_20s', 'age_30s', 'age_40s', 'age_50s', 'age_60s', 'age_70s',
       'age_over_80', 'male', 'female', 'married', 'divorced', 'never_married',
       'widowed', 'family_size', 'family_dual_income',
       'income_household_median', 'income_household_under_5',
       'income_household_5_to_10', 'income_household_10_to_15',
       'income_household_15_to_20', 'income_household_20_to_25',
       'income_household_25_to_35', 'income_household_35_to_50',
       'income_household_50_to_75', 'income_household_75_to_100',
       'income_household_100_to_150', 'income_household_150_over',
       'income_household_six_figure', 'income_individual_median',
       'home_ownership', 'housing_units', 'home_value', 'rent_median',
       'rent_burden', 'education_less_highschool', 'education_highschool',
       'education_some_college', 'education_bachelors', 'education_graduate',
       'education_college_or_above', 'education_stem_degree',
       'labor_force_participation', 'unemployment_rate', 'self_employed',
       'farmer', 'race_white', 'race_black', 'race_asian', 'race_native',
       'race_pacific', 'race_other', 'race_multiple', 'hispanic', 'disabled',
       'poverty', 'limited_english', 'commute_time', 'health_uninsured',
       'veteran', 'Ozone', 'PM25', 'N02']

In [ ]:
def get_accuracy(classifier, X_train, y_train, X_val, y_val):
  classifier.fit(X_train, y_train)
  y_pred = classifier.predict(X_val)
  accuracy = accuracy_score(y_val, y_pred)
  return accuracy

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler

def preprocess_data(X, categorical_features, numerical_features):
    # Impute missing values in numerical features with mean, then scale
    numerical_transformer = Pipeline([
        ("imputer", SimpleImputer(strategy="mean")),
        ("scaler", StandardScaler())
    ])

    # Encode categorical features
    categorical_transformer = OneHotEncoder(handle_unknown="ignore")

    # Column transformer for preprocessing
    preprocessor = ColumnTransformer([
        ("num", numerical_transformer, numerical_features),
        ("cat", categorical_transformer, categorical_features)
    ])

    # Fit the transformer and transform X
    X_transformed = preprocessor.fit_transform(X)

    # Get numerical feature names
    num_feature_names = numerical_features  # Numerical features remain the same

    # Retrieve fitted OneHotEncoder and get categorical feature names
    fitted_ohe = preprocessor.named_transformers_["cat"]
    cat_feature_names = fitted_ohe.get_feature_names_out(categorical_features)

    # Map categorical features to their one-hot encoded indices
    cat_feature_groups = {}
    start_idx = len(num_feature_names)  # Start after numerical features
    for i, cat_feature in enumerate(categorical_features):
        num_categories = sum(1 for name in cat_feature_names if name.startswith(cat_feature))
        cat_feature_groups[cat_feature] = list(range(start_idx, start_idx + num_categories))
        start_idx += num_categories

    return X_transformed, preprocessor, num_feature_names, cat_feature_groups


In [ ]:
from sklearn.model_selection import cross_val_score

def sequential_feature_selection(X, y, classifier, categorical_features, numerical_features, cv=5, scoring='accuracy'):
    # Preprocess data (apply imputation, encoding, and scaling)
    X_transformed, preprocessor, num_feature_names, cat_feature_groups = preprocess_data(X, categorical_features, numerical_features)

    selected_features = []
    selected_feature_names = []
    remaining_features = {feature: [feature] for feature in num_feature_names}  # Individual numerical features
    remaining_features.update(cat_feature_groups)  # Grouped categorical features
    best_score = 0  # Track best accuracy

    while remaining_features:
        best_feature = None
        best_new_score = best_score

        for feature, feature_indices in remaining_features.items():
            current_features = selected_features + feature_indices
            X_subset = X_transformed[:, current_features]

            # Perform cross-validation
            scores = cross_val_score(classifier, X_subset, y, cv=cv, scoring=scoring)
            mean_score = np.mean(scores)

            if mean_score > best_new_score:
                best_new_score = mean_score
                best_feature = feature

        if best_feature is not None:
            selected_features.extend(remaining_features[best_feature])
            selected_feature_names.append(best_feature)
            del remaining_features[best_feature]
            best_score = best_new_score
            print(f"Added feature: {best_feature}, New Accuracy: {best_score:.4f}")
        else:
            break  # Stop if no improvement

    return selected_feature_names

In [ ]:
svc_classifier = SVC(C=1.0, kernel='linear', gamma='scale', random_state=0)

selected_features, selected_feature_names = sequential_feature_selection(df, df[target_column], svc_classifier, selected_features_categorical, selected_features_numerical)

print(selected_features)
print(selected_feature_names)

UFuncTypeError: ufunc 'maximum' did not contain a loop with signature matching types (dtype('<U11'), dtype('<U11')) -> None

In [ ]:
decision_tree_classifier = DecisionTreeClassifier(min_samples_leaf=50, max_depth=15, random_state=0)

selected_features, selected_feature_names = sequential_feature_selection(df, df[target_column[0]], decision_tree_classifier, selected_features_categorical, selected_features_numerical)

print(selected_features)
print(selected_feature_names)

In [ ]:
df.columns

In [ ]:
X_train